# Import and Set

In [1]:
import sys
sys.path.append('../')
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from utils import *
from models.Transformers import *

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
stock_symbol = '5871.TW'
end_date = '2024-12-31'

"""
Trials:
    [decoder = {True, False}, num_class = {1, 2}]
Progress:
    [decoder = True, num_class = 1]
Pendng:
Finished:
    [decoder = False, num_class = 1] [decoder = True, num_class = 1]
"""
num_class = 2
init = True
fp16_training = True
num_epochs = 500
config = {
    'lr': 0.001,
}

# Data

In [2]:
if num_class == 1:
    with open('../DataLoader/dataloader_1.pk', 'rb') as f:
        data = pickle.load(f)
    trainloader = data['trainloader']
    validloader = data['validloader']
    # dataloader_test = data['testloader']
else:
    with open('../DataLoader/dataloader.pk', 'rb') as f:
        data = pickle.load(f)
    trainloader = data['trainloader']
    validloader = data['validloader']
    # dataloader_test = data['testloader']
with open('../DataLoader/src.pk', 'rb') as f:
    src = pickle.load(f)
    src = src.to(device)

for x, y in trainloader:
    print(x.shape)
    break
batch_size = x.size(0)

torch.Size([32, 6, 100])


# Setting

- Model, Criteria, Optimizer, Fp16, Previous Tarin Inofrmation

In [3]:
"""
Choose if fp16 and define model
pip install accelerate==0.2.0
"""
# Model
if fp16_training:
    print('Accelerating')
    from accelerate import Accelerator
    accelerator = Accelerator()
    device = accelerator.device
    model = TransformerEncoderDecoder(num_class=num_class)
else:
    model = TransformerEncoderDecoder(num_class=num_class).to(device)
        
Model = model.model_type # Model name

"""
Init for models, learning rate, ...
"""
# Check path
if os.path.exists(f'Temp//{Model}_{stock_symbol}_LastTrainInfo.pk'):
    # Check Init
    if init:
        print("Init model")
        lr = config['lr']
        last_epoch = 0
        min_val_loss = 10000
        loss_train = []
        loss_valid = []
    else:
        print('Load from last train epoch')
        with open(f'Temp//{Model}_class{num_class}_{stock_symbol}_LastTrainInfo.pk', 'rb') as f:
            last_train_info = pickle.load(f)
        lr = last_train_info['lr']
        last_epoch = last_train_info['epoch']
        min_val_loss = last_train_info['min val loss']
        model.load_state_dict(torch.load(f'Temp//{Model}_class{num_class}_{stock_symbol}_checkpoint_LastTrainModel.pt'))
        with open(f'Temp//{Model}_class{num_class}_{stock_symbol}_TrainValHistLoss.pk', 'rb') as f:
            loss_train_val = pickle.load(f)
        loss_train = loss_train_val['train']
        loss_valid = loss_train_val['valid']
else:
    print("Init model")
    lr = config['lr']
    last_epoch = 0
    min_val_loss = 10000.0
    loss_train = []
    loss_valid = []
print(f'Last train epoch: {last_epoch}  '
        f'Last train lr: {lr}   '
        f'Min val loss: {min_val_loss}')

# Criterion and Optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=0.00001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=len(trainloader)*1, gamma=0.9)        

# Prepare
if fp16_training:
    print('Accelerate Prepare')    
    model, optimizer, trainloader, validloader, scheduler = \
        accelerator.prepare(model, optimizer, trainloader, validloader, scheduler)
        
# Check device
for name, param in model.named_parameters():
    print(f"Parameter '{name}' is on device: {param.device}")
    break

Accelerating
Init model
Last train epoch: 0  Last train lr: 0.001   Min val loss: 10000.0
Accelerate Prepare
Parameter 'embedding.weight' is on device: cuda:0


# Train

In [4]:
"""
--- Original ---------
batch_x: (batch_size, d_model, seqlen) 
src: (total_length, d_model, seq_len)
--- Input of model ---
batch_x: (batch_size, seq_len, d_model) -> use src.permute()
src: (total_length, seq_len, d_model)   -> use batch.permute()
"""
src = src.permute(0, 2, 1)
for epoch in range(last_epoch, num_epochs):
    # Training phase
    model.train()
    loss_train_e = 0
    for batch_x, batch_y in tqdm(trainloader): 
        if not fp16_training:
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)    
               
        batch_x = batch_x.permute(0, 2, 1)
        optimizer.zero_grad()
        
        if batch_x.size(0) != batch_size:
            continue
        memory, outputs = model(src=src, tgt=batch_x, train=True)
    
        # Loss
        loss = criterion(outputs, batch_y)
        if fp16_training:
            accelerator.backward(loss)
        else:
            loss.backward()
        optimizer.step()
        if epoch > 50:
            scheduler.step()
        loss_train_e += loss.item()
        
    loss_train_e /= len(trainloader)
    loss_train.append(loss_train_e)
    
    loss_valid_e = 0
    with torch.no_grad():
        model.eval()
        for batch_x_val, batch_y_val in tqdm(validloader):
            # batch_x_val = mask(batch_x_val)
            if not fp16_training:
                batch_x_val = batch_x_val.to(device)
                batch_y_val = batch_y_val.to(device)
            batch_x_val = batch_x_val.permute(0, 2, 1)
            
            if batch_x_val.size(0) != batch_size:
                    continue
            memory, outputs_val = model(src, batch_x_val, False, memory)
                
            loss = criterion(outputs_val, batch_y_val)
            loss_valid_e += loss.item()
        loss_valid_e /= len(validloader)
        loss_valid.append(loss_valid_e)
            
        torch.save(model.state_dict(), f'Temp/{Model}_class{num_class}_{stock_symbol}_checkpoint_LastTrainModel.pt')
        if loss_valid_e < min_val_loss:
            min_val_loss = loss_valid_e
            print(f'New best model found in epoch {epoch} with val loss: {min_val_loss}')
            torch.save(model.state_dict(), f'Model_Result/{Model}_class{num_class}_{stock_symbol}_best_model.pt')            
        if epoch % 50 == 0:
            pass
            # torch.save(model, f'ConformerResult/Conformerr_{stock_symbol}_checkpoint_{epoch}.pt')
            
    with open(f'Temp/{Model}_class{num_class}_{stock_symbol}_TrainValHistLoss.pk', 'wb') as f:
        pickle.dump({'train': loss_train, 'valid': loss_valid}, f)
    with open(f'Temp/{Model}_class{num_class}_{stock_symbol}_LastTrainInfo.pk', 'wb') as f:
        pickle.dump({'min val loss': min_val_loss, 'epoch': epoch, 'lr': optimizer.param_groups[0]['lr']}, f)
        
    # Print statistics
    print(f'Epoch [{epoch}/{num_epochs}]',
        f'Training Loss: {loss_train_e:.10f}',
        f'Valid Loss: {loss_valid_e:.10f}')

100%|██████████| 16/16 [00:00<00:00, 64.79it/s]


Epoch [413/500] Training Loss: 2.6722880322 Valid Loss: 4.2389761657


100%|██████████| 16/16 [00:00<00:00, 87.04it/s]


Epoch [414/500] Training Loss: 2.5684217084 Valid Loss: 4.2375257462


100%|██████████| 16/16 [00:00<00:00, 95.76it/s]


...

Epoch [497/500] Training Loss: 2.5280138357 Valid Loss: 4.2390164509


100%|██████████| 16/16 [00:00<00:00, 104.01it/s]


Epoch [498/500] Training Loss: 2.4703211738 Valid Loss: 4.2376339585


100%|██████████| 16/16 [00:00<00:00, 99.06it/s] 


Epoch [499/500] Training Loss: 2.5029005175 Valid Loss: 4.2399305180
